In [1]:
!pip install pyspark==3.5.0 findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=86c22e3f5f6b071d555cacc0c5a0d337e16a541124c4af20756b01815b38538e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import findspark
findspark.init()

In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Day16").getOrCreate()
sc = spark.sparkContext

In [23]:

ratings_df = spark.read.csv("sample_data/ratings.csv", header=True, schema = ratings_schema)
ratings_df.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    110|   1.0|1425941529|
|     1|    147|   4.5|1425942435|
|     1|    858|   5.0|1425941523|
|     1|   1221|   5.0|1425941546|
|     1|   1246|   5.0|1425941556|
+------+-------+------+----------+
only showing top 5 rows



In [25]:
ratings_rdd = sc.parallelize("sample_data/ratings.csv")

# Calculate average rating per movie
average_ratings = ratings_rdd.map(lambda x: (x[1], (x[2], 1))) \
 .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
 .map(lambda x: (x[0], x[1][0] / x[1][1]))

# Filter for movies with average rating above 4.0
top_rated_movies = average_ratings.filter(lambda x: x[1] > 4.0)